## 用技術指標預測 (昨天預測今天)

In [1]:
import numpy as np
import pandas as pd

import datetime as datetime
import talib

In [2]:
company = '台積電'

In [3]:
df2018 = pd.read_csv("bda2019_dataset/TWSE2018.csv", encoding="utf-8")
df2017 = pd.read_csv("bda2019_dataset/TWSE2017.csv", encoding="utf-8")
df2016 = pd.read_csv("bda2019_dataset/TWSE2016.csv", encoding="utf-8")
df_all = pd.concat([df2018,df2017,df2016],axis=0, ignore_index=True)
df_company=df_all[df_all['證券代碼'].str.contains(company)]
df_company=df_company.reset_index(drop=True)
df_company

,證券代碼,年月日,開盤價(元),最高價(元),最低價(元),收盤價(元),成交量(千股),成交值(千元),成交筆數(筆),流通在外股數(千股),本益比-TSE,股價淨值比-TSE
0,2330 台積電,2018/12/28,223.5,225.5,222.5,225.5,"30,156","6,762,447","10,247","25,930,380",16.68,3.71
1,2330 台積電,2018/12/27,222,224,220.5,223,"21,915","4,874,845","9,531","25,930,380",16.49,3.67
2,2330 台積電,2018/12/26,218.5,219.5,216,216.5,"16,893","3,678,108","6,998","25,930,380",16.01,3.56
3,2330 台積電,2018/12/25,215.5,218,215,217.5,"19,402","4,198,767","8,065","25,930,380",16.09,3.58
4,2330 台積電,2018/12/24,221.5,221.5,219.5,220,"23,376","5,151,549","6,122","25,930,380",16.27,3.62
...,...,...,...,...,...,...,...,...,...,...,...,...
732,2330 台積電,2016/1/8,118.6,121.29,118.6,120.4,"52,641","7,022,206","11,784","25,930,380",11.07,3.02
733,2330 台積電,2016/1/7,120.85,121.29,117.25,119.5,"63,475","8,432,791","19,061","25,930,380",10.99,3.00
734,2330 台積電,2016/1/6,123.99,123.99,121.29,121.74,"53,873","7,321,139","15,926","25,930,380",11.2,3.05
735,2330 台積電,2016/1/5,124.89,125.79,123.09,123.99,"46,502","6,436,850","15,836","25,930,380",11.4,3.11


In [4]:
list2330 = list()
for row in df_company.index:
    date=df_company.loc[row,'年月日']
    opens=float(df_company.loc[row,'開盤價(元)'])
    close=float(df_company.loc[row,'收盤價(元)'])
    high=float(df_company.loc[row,'最高價(元)'])
    low=float(df_company.loc[row,'最低價(元)'])
    volumn=int(df_company.loc[row,'成交量(千股)'].replace(',',''))
    list2330.append((date,opens,close,high,low,volumn))

In [5]:
list2330.reverse()
df_2330 = pd.DataFrame(list2330, columns=['Date','Open','Close','High','Low','Volume'])
df_2330

,Date,Open,Close,High,Low,Volume
0,2016/1/4,128.03,125.34,128.93,124.89,43800
1,2016/1/5,124.89,123.99,125.79,123.09,46502
2,2016/1/6,123.99,121.74,123.99,121.29,53873
3,2016/1/7,120.85,119.50,121.29,117.25,63475
4,2016/1/8,118.60,120.40,121.29,118.60,52641
...,...,...,...,...,...,...
732,2018/12/24,221.50,220.00,221.50,219.50,23376
733,2018/12/25,215.50,217.50,218.00,215.00,19402
734,2018/12/26,218.50,216.50,219.50,216.00,16893
735,2018/12/27,222.00,223.00,224.00,220.50,21915


In [6]:
#算出技術指標
sma_5 = talib.SMA(df_2330['Close'], 5)
sma_10 = talib.SMA(df_2330['Close'], 10)
k_value, d_value = talib.STOCH(df_2330['High'], df_2330['Low'], df_2330['Close'])
momentum = talib.MOM(df_2330['Close'], timeperiod=1)
roc = talib.ROC(df_2330['Close'], timeperiod=1)
william = talib.WILLR(df_2330['High'], df_2330['Low'], df_2330['Close'], timeperiod=14)
ad = talib.AD(df_2330['High'], df_2330['Low'], df_2330['Close'], df_2330['Volume'])
ema_8 = talib.EMA(df_2330['Close'], timeperiod=8)
macd, macd_signal, macd_hist = talib.MACD(df_2330['Close'], fastperiod=12, slowperiod=26, signalperiod=9)
rsi = talib.RSI(df_2330['Close'], timeperiod=14)
obv = talib.OBV(df_2330['Close'], df_2330['Volume'])
upperband, middleband, lowerband = talib.BBANDS(df_2330['Close'], timeperiod=21, nbdevup=2, nbdevdn=2, matype=0)

df_2330['sma_5'] = sma_5
df_2330['sma_10'] = sma_10
df_2330['k'] = k_value
df_2330['d'] = d_value
df_2330['momentum'] = momentum
df_2330['rate of change'] = roc
df_2330['william'] = william
df_2330['a/d'] = ad
df_2330['ema_8'] = ema_8
df_2330['macd'] = macd
df_2330['macd_signal'] = macd_signal
df_2330['macd_hist'] = macd_hist
df_2330['rsi'] = rsi
df_2330['obv'] = obv
# df_2330['bb_upperband'] = upperband
# df_2330['bb_middleband'] = middleband
# df_2330['bb_lowerband'] = lowerband
df_2330.head(40)

,Date,Open,Close,High,Low,Volume,sma_5,sma_10,k,d,momentum,rate of change,william,a/d,ema_8,macd,macd_signal,macd_hist,rsi,obv
0,2016/1/4,128.03,125.34,128.93,124.89,43800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-34042.574257,NaN,NaN,NaN,NaN,NaN,43800.0
1,2016/1/5,124.89,123.99,125.79,123.09,46502,NaN,NaN,NaN,NaN,-1.35,-1.077070,NaN,-49543.240924,NaN,NaN,NaN,NaN,NaN,-2702.0
2,2016/1/6,123.99,121.74,123.99,121.29,53873,NaN,NaN,NaN,NaN,-2.25,-1.814662,NaN,-85458.574257,NaN,NaN,NaN,NaN,NaN,-56575.0
3,2016/1/7,120.85,119.50,121.29,117.25,63475,NaN,NaN,NaN,NaN,-2.24,-1.839987,NaN,-78231.222772,NaN,NaN,NaN,NaN,NaN,-120050.0
4,2016/1/8,118.60,120.40,121.29,118.60,52641,122.194,NaN,NaN,NaN,0.90,0.753138,NaN,-60423.300839,NaN,NaN,NaN,NaN,NaN,-67409.0
5,2016/1/11,119.50,119.50,120.85,117.25,48536,121.026,NaN,NaN,NaN,-0.90,-0.747508,NaN,-48289.300839,NaN,NaN,NaN,NaN,NaN,-115945.0
6,2016/1/12,119.50,119.50,120.40,118.15,35133,120.128,NaN,NaN,NaN,0.00,0.000000,NaN,-41262.700839,NaN,NaN,NaN,NaN,NaN,-115945.0
7,2016/1/13,119.95,119.95,121.74,119.50,50237,119.770,NaN,NaN,NaN,0.45,0.376569,NaN,-71315.191911,121.240000,NaN,NaN,NaN,NaN,-65708.0
8,2016/1/14,117.25,118.15,118.60,117.25,39498,119.500,NaN,37.853654,35.569173,-1.80,-1.500625,NaN,-58149.191911,120.553333,NaN,NaN,NaN,NaN,-105206.0
9,2016/1/15,123.54,123.09,123.99,121.74,79648,120.038,121.116,55.608353,44.472116,4.94,4.181126,NaN,-42219.591911,121.117037,NaN,NaN,NaN,NaN,-25558.0


In [7]:
#把有NA的資料刪掉
df_2330 = df_2330.drop(columns=['Open','High','Low','Volume'])
df_2330 = df_2330[33:]
df_2330 = df_2330.reset_index(drop=True)
df_2330.head()

,Date,Close,sma_5,sma_10,k,d,momentum,rate of change,william,a/d,ema_8,macd,macd_signal,macd_hist,rsi,obv
0,2016/2/26,133.87,134.41,133.917,39.271255,42.959392,-1.35,-0.998373,-30.493708,336972.617765,133.727349,3.671588,3.813942,-0.142354,59.107660,236126.0
1,2016/3/1,136.57,134.59,134.546,66.599190,49.392713,2.70,2.016882,-4.550051,381018.617765,134.359049,3.710158,3.793185,-0.083027,64.196526,280172.0
2,2016/3/2,137.47,135.22,135.040,72.107990,59.326145,0.90,0.659003,-11.472603,344917.617765,135.050371,3.769890,3.788526,-0.018636,65.727569,316273.0
3,2016/3/3,138.37,136.30,135.580,90.851635,76.519605,0.90,0.654688,-3.918077,354827.184929,135.788067,3.845523,3.799925,0.045597,67.236393,345140.0
4,2016/3/4,137.02,136.66,135.715,80.634741,81.198122,-1.35,-0.975645,-15.939448,354827.184929,136.061830,3.753263,3.790593,-0.037330,62.772258,307346.0


In [8]:
# 正規化(scale)
from sklearn import preprocessing

df_2330['a/d'] = preprocessing.scale(df_2330['a/d'])
df_2330['obv'] = preprocessing.scale(df_2330['obv'])
df_2330.head(10)

,Date,Close,sma_5,sma_10,k,d,momentum,rate of change,william,a/d,ema_8,macd,macd_signal,macd_hist,rsi,obv
0,2016/2/26,133.87,134.410,133.917,39.271255,42.959392,-1.35,-0.998373,-30.493708,-2.775705,133.727349,3.671588,3.813942,-0.142354,59.107660,-2.422069
1,2016/3/1,136.57,134.590,134.546,66.599190,49.392713,2.70,2.016882,-4.550051,-2.668442,134.359049,3.710158,3.793185,-0.083027,64.196526,-2.294656
2,2016/3/2,137.47,135.220,135.040,72.107990,59.326145,0.90,0.659003,-11.472603,-2.756357,135.050371,3.769890,3.788526,-0.018636,65.727569,-2.190226
3,2016/3/3,138.37,136.300,135.580,90.851635,76.519605,0.90,0.654688,-3.918077,-2.732225,135.788067,3.845523,3.799925,0.045597,67.236393,-2.106722
4,2016/3/4,137.02,136.660,135.715,80.634741,81.198122,-1.35,-0.975645,-15.939448,-2.732225,136.061830,3.753263,3.790593,-0.037330,62.772258,-2.216049
5,2016/3/7,136.57,137.200,135.805,73.418876,81.635084,-0.45,-0.328419,-27.722772,-2.743940,136.174756,3.602309,3.752936,-0.150627,61.310974,-2.285625
6,2016/3/8,136.57,137.200,135.895,59.300744,71.118120,0.00,0.000000,-33.283804,-2.644964,136.262588,3.442989,3.690947,-0.247957,61.310974,-2.285625
7,2016/3/9,137.47,137.200,136.210,59.569606,64.096409,0.90,0.659003,-19.910847,-2.584974,136.530902,3.350724,3.622902,-0.272178,63.293025,-2.214366
8,2016/3/10,138.81,137.288,136.794,73.422420,64.097590,1.34,0.974758,-0.000000,-2.508646,137.037368,3.347147,3.567751,-0.220604,66.079388,-2.123699
9,2016/3/11,139.26,137.736,137.198,90.051967,74.347998,0.45,0.324184,-0.000000,-2.432430,137.531286,3.342097,3.522620,-0.180523,66.985712,-2.033165


In [9]:
#判斷這筆資料是漲還是跌
df_2330['next_mom'] = df_2330['momentum'].shift(periods=-1) #把明天的Momentum移到今天，正的代表今天到明天是漲，負的代表是跌
df_2330['next_mom'].fillna(value=0.0, inplace=True)
df_2330['label']='平'
df_2330.loc[df_2330['next_mom']>0.0, 'label'] = '漲'
df_2330.loc[df_2330['next_mom']<0.0, 'label'] = '跌'
df_2330.tail(10)

,Date,Close,sma_5,sma_10,k,d,momentum,rate of change,william,a/d,ema_8,macd,macd_signal,macd_hist,rsi,obv,next_mom,label
694,2018/12/18,222.5,224.2,222.95,43.572985,59.259259,-1.0,-0.447427,-75.757576,1.198622,223.641537,-1.797317,-2.035121,0.237803,45.362055,1.386169,3.0,漲
695,2018/12/19,225.5,224.0,222.90,53.202614,52.338417,3.0,1.348315,-57.575758,1.255643,224.054529,-1.545576,-1.937212,0.391636,49.412294,1.453902,-4.5,跌
696,2018/12/20,221.0,223.0,223.00,41.616162,46.130587,-4.5,-1.995565,-84.848485,1.161407,223.375745,-1.689704,-1.887710,0.198006,44.128114,1.341962,2.5,漲
697,2018/12/21,223.5,223.2,223.25,53.581974,49.466917,2.5,1.131222,-68.750000,1.252869,223.403357,-1.583939,-1.826956,0.243017,47.487919,1.477767,-2.0,跌
698,2018/12/22,221.5,222.8,223.50,41.958042,45.718726,-2.0,-0.894855,-66.666667,1.265495,222.980389,-1.642568,-1.790078,0.147511,45.148854,1.462768,-1.5,跌
699,2018/12/24,220.0,222.3,223.25,41.025641,45.521886,-1.5,-0.677201,-83.333333,1.237032,222.318080,-1.789441,-1.789951,0.000510,43.421392,1.395148,-2.5,跌
700,2018/12/25,217.5,220.7,222.35,27.207977,36.730553,-2.5,-1.136364,-80.000000,1.268531,221.247396,-2.083551,-1.848671,-0.234880,40.631074,1.339023,-1.0,跌
701,2018/12/26,216.5,219.8,221.40,19.943020,29.392213,-1.0,-0.459770,-88.000000,1.239147,220.192419,-2.370007,-1.952938,-0.417069,39.536628,1.290156,6.5,漲
702,2018/12/27,223.0,219.7,221.45,44.444444,30.531814,6.5,3.002309,-36.000000,1.262019,220.816326,-2.048911,-1.972133,-0.076778,49.128596,1.353550,2.5,漲
703,2018/12/28,225.5,220.5,221.65,68.518519,44.301994,2.5,1.121076,-16.000000,1.335456,221.857142,-1.574560,-1.892618,0.318058,52.265214,1.440783,0.0,平


In [10]:
print('原先資料量: %d' % len(df_2330))
df_2330 = df_2330[df_2330['label'].str.contains('漲|跌')]
df_2330=df_2330.reset_index(drop=True)
df_2330_1 = df_2330[df_2330['label'].str.contains('漲')]
df_2330_2 = df_2330[df_2330['label'].str.contains('跌')]
print('漲跌資料量: %d' % len(df_2330))
print('漲的資料量: %d' % len(df_2330_1))
print('跌的資料量: %d' % len(df_2330_2))

原先資料量: 704
漲跌資料量: 631
漲的資料量: 346
跌的資料量: 285


In [11]:
df_2330.iloc[507,0]

'2018/7/2'

In [12]:
df_2330 = df_2330.drop(columns=['Date','Close','next_mom'])
df_2330_train = df_2330[:507]
df_2330_test = df_2330[507:]
print(len(df_2330_train), len(df_2330_test))

507 124


In [13]:
df_2330_train_1 = df_2330_train.drop(columns=['label'])
df_2330_test_1 = df_2330_test.drop(columns=['label'])
X_train = df_2330_train_1.to_numpy()
X_test = df_2330_test_1.to_numpy()
Y = df_2330['label']

In [14]:
len(Y)

631

In [15]:
# 跌是[0,1] 漲是[1,0]
from sklearn.preprocessing import LabelEncoder,OneHotEncoder

le = LabelEncoder()
Y_label= le.fit_transform(Y).reshape(-1,1)
ohe = OneHotEncoder()
Y_label = ohe.fit_transform(Y_label).toarray()
Y_label

C:\Users\Daniel\Anaconda3\envs\tfenv\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [0., 1.],
       [1., 0.],
       [1., 0.]])

In [16]:
Y_train = Y_label[:507]
Y_test = Y_label[507:]
print(len(Y_train),len(Y_test))

507 124


In [17]:
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(507, 14)
(124, 14)
(507, 2)
(124, 2)


In [18]:
from keras.models import Model, Sequential
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding, SpatialDropout1D, Flatten, Conv1D, MaxPooling1D, TimeDistributed
from keras.optimizers import RMSprop, adam
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.callbacks import EarlyStopping

Using TensorFlow backend.


In [19]:
model = Sequential()
model.add(Dense(64, input_shape=(14,), activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='RMSprop', metrics=['accuracy'])
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 64)                960       
_________________________________________________________________
dense_2 (Dense)              (None, 32)                2080      
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 66        
Total params: 3,106
Trainable params: 3,106
Non-trainable params: 0
_________________________________________________________________


In [20]:
history = model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=100, batch_size=16, verbose=1)

Instructions for updating:
Use tf.cast instead.
Train on 507 samples, validate on 124 samples
Epoch 1/100
507/507 [==============================] - 2s 4ms/step - loss: 6.0341 - acc: 0.4911 - val_loss: 6.2329 - val_acc: 0.4355
Epoch 2/100
507/507 [==============================] - 0s 216us/step - loss: 4.9120 - acc: 0.4655 - val_loss: 1.4593 - val_acc: 0.5565
Epoch 3/100
507/507 [==============================] - 0s 223us/step - loss: 2.2898 - acc: 0.5266 - val_loss: 0.9557 - val_acc: 0.4032
Epoch 4/100
507/507 [==============================] - 0s 212us/step - loss: 0.8082 - acc: 0.5148 - val_loss: 0.7115 - val_acc: 0.5000
Epoch 5/100
507/507 [==============================] - 0s 216us/step - loss: 0.7254 - acc: 0.5108 - val_loss: 0.7010 - val_acc: 0.5081
Epoch 6/100
507/507 [==============================] - 0s 222us/step - loss: 0.6965 - acc: 0.5306 - val_loss: 0.6848 - val_acc: 0.5645
Epoch 7/100
507/507 [==============================] - 0s 216us/step - loss: 0.6942 - acc: 0.5404 